In [9]:
import os
import pandas as pd
import numpy as np
from glob import glob
import tensorflow.keras.backend as K

# --- Define metrics ---
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def madl(y_true, y_pred):
    return np.mean((-1)*np.sign(y_true*y_pred)*np.abs(y_true))


def madl_custom(y_true, y_pred):
    y_true = K.cast(y_true, K.floatx())
    y_pred = K.cast(y_pred, K.floatx())
    
    direction_mismatch = K.cast(K.less(y_true * y_pred, 0), K.floatx())  # 1 if opposite sign
    return K.mean(K.abs(y_true - y_pred) * direction_mismatch)


In [10]:
def analyze_csv_files(folder="."):
    results = []
    csv_files = glob(os.path.join(folder, "*.csv"))

    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            if df.shape[1] != 3:
                print(f"❌ Skipping {file_path}: expected 3 columns, found {df.shape[1]}")
                continue
            
            # Assuming structure: [index, y_true, y_pred]
            y_true = df.iloc[:, 1].values.astype(float)
            y_pred = df.iloc[:, 2].values.astype(float)
            
            metrics = {
                "file": os.path.basename(file_path),
                "MAPE": mape(y_true, y_pred),
                "MSE": mse(y_true, y_pred),
                "MAE": mae(y_true, y_pred),
                "MADL_Loss": madl(y_true, y_pred),
                "MADL_Custom": madl_custom(y_true, y_pred)
            }
            results.append(metrics)
            print(f"✅ Processed {file_path}")
        except Exception as e:
            print(f"⚠️ Error processing {file_path}: {e}")
    
    if results:
        results_df = pd.DataFrame(results)
        print("\n=== METRICS SUMMARY ===")
        print(results_df.to_string(index=False))
        
        # Optional: save results
        results_df.to_csv("metrics_summary.csv", index=False)
        print("\n📁 Saved results to 'metrics_summary.csv'")
        return results_df
    else:
        print("No valid CSV files found.")
        return pd.DataFrame()

# --- Run ---
if __name__ == "__main__":
    analyze_csv_files(".")

✅ Processed .\7inputs_standardsc_gmadl.csv
✅ Processed .\7inputs_standardsc_gmadl_v2.csv
❌ Skipping .\apl_data.csv: expected 3 columns, found 6
❌ Skipping .\best_hyperparameters_all_windows.csv: expected 3 columns, found 22
❌ Skipping .\best_hyperparameters_all_windows_7inputs_standardsc_gmadl.csv: expected 3 columns, found 22
❌ Skipping .\data_vix.csv: expected 3 columns, found 14
❌ Skipping .\final_settle_df.csv: expected 3 columns, found 5
✅ Processed .\hybrid_5inputs_standard_sc.csv
✅ Processed .\hybrid_5inputs_standard_sc_v2.csv
✅ Processed .\hybrid_7inputs.csv
✅ Processed .\hybrid_7inputs_standard_sc.csv
✅ Processed .\hybrid_7inputs_standard_sc_custom_corr.csv
✅ Processed .\hybrid_7inputs_standard_sc_custom_madl_v2.csv
✅ Processed .\hybrid_7inputs_standard_sc_madl.csv
✅ Processed .\hybrid_7inputs_standard_sc_madl_mse.csv
✅ Processed .\hybrid_7inputs_standard_sc_madl_v2.csv
✅ Processed .\hybrid_7inputs_standard_sc_madl_v3.csv
✅ Processed .\hybrid_baseline.csv
✅ Processed .\lstm_ba